In [ ]:
from typing import List
import openai


def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "あなたは航空会社のカスタマーサポートエージェントです。"
    }
    messages = [system_message] + messages
    completion = openai.chat.completions.create(
        messages=messages,
        model="gpt-4o-mini"
    )
    return completion.choices[0].message.model_dump()

In [ ]:
my_chat_bot([{"role": "user", "content": "こんにちは!"}])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI


system_prompt_template = """あなたは航空会社の顧客です。
あなたはカスタマーサポート担当者とやり取りしています。

{instructions}

会話が終了したら、単語「FINISHED」で応答してください。"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages")
    ]
)

instructions =  """あなたの名前は Harrison です。あなたはアラスカへの旅行の払い戻しを求めています。
すべてのお金を返してもらいたいと思っています。
この旅行は5年前に行われました。"""


prompt = prompt.partial(name="Harrison", instructions=instructions)

model = ChatOpenAI()

simulated_user = prompt | model

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="こんにちは！どのようなご用件でしょうか？お手伝いできることがあれば教えてください。")]
simulated_user.invoke({"messages": messages})

In [ ]:
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage


def chat_bot_node(state):
    messages = state["messages"]
    messages = [convert_message_to_dict(m) for m in messages]
    chat_bot_response = my_chat_bot(messages)
    return {"messages": [AIMessage(content=chat_bot_response["content"])]}

In [ ]:
# 人間役のLLMはノードの役割を入れ替えて渡す必要がある
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages

In [ ]:
def simulated_user_node(state):
    messages = state["messages"]

    # 役割を入れ替える
    new_messages = _swap_roles(messages)

    response = simulated_user.invoke({"messages": new_messages})

    # responseの出力はAIMessageであるが人間役であるためHumanMessageに変換する
    return {"messages": [HumanMessage(content=response.content)]}

In [ ]:
def should_continue(state):
    messages = state["messages"]
    if len(messages) > 6:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    {"end": END, "continue": "chat_bot"}
)
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

for chunk in simulation.stream({"messages": []}):
    if END not in chunk:
        print(chunk)
        print("-----")

In [ ]:
from IPython.display import Image, display

display(Image(simulation.get_graph().draw_mermaid_png()))
